# 4 训练Transformer LM
我们现在已经有了通过分词器预处理数据和模型（Transformer）的步骤。接下来需要完成支持训练的所有代码，主要包括以下部分：
* 损失函数：需要定义损失函数（交叉熵）。
* 优化器：需要定义用于最小化该损失的优化器（AdamW）。
* 训练循环：需要构建所有支持性的基础设施，包括加载数据、保存检查点以及管理训练过程。

## 4.1 交叉熵损失

[“交叉熵”如何做损失函数？打包理解“信息量”、“比特”、“熵”、“KL散度”、“交叉熵”](https://www.bilibili.com/video/BV15V411W7VB/?share_source=copy_web&vd_source=c379ccdab784832c917bb852fa2b0584)

长度为 $m$ 的token序列组成一个训练集 $D$，此时定义分布概率 $p_\theta (x_{i+1} \mid x_{1:i})$ 表示在给定序列 $x$ 的前 $i$ 个元素 $x_{1:i}$ 条件下，模型预测下一个元素 $x_{i+1}$ 的概率。

我们定义标准的交叉熵（负对数似然）损失函数：
$$\ell (\theta ; D)=\frac{1}{|D|m}\sum_{x \in D}\sum_{i=1}^{m}-\log p_\theta (x_{i+1} \mid x_{1:i})$$

* $\frac{1}{|D|m}$ 是一个归一化因子。其中 $|D|$ 表示训练集中序列 $D$ 的数量，$m$ 表示每个序列的长度。通过除以 $|D|m$，可以将整个训练集上的损失进行平均，使得不同规模训练集的损失具有可比性。

（注意：Transformer 的一次前向传播可以得到所有 $i$ 对应的 $p_{\theta}(x_{i+1} \mid x_{1:i})$ 。）

具体的：
$$
p(x_{i+1} \mid x_{1:i}) = \text{softmax}(o_i)[x_{i+1}] = \frac{\exp(o_i[x_{i+1}])}{\sum_{a=1}^{\text{vocab\_ size}} \exp(o_i[a])}
$$

* Transfomer 为每个位置计算 logits 向量 $o_i \in \mathbb{R}^{\text{vocab\_size}}$
* $\text{softmax}(o_i)[x_{i+1}]$ 表示 softmax 作用于 $o_i$ 向量，并取向量中的 $x_{i+1}$ 对应的概率值


实现交叉熵损失时需要特别注意数值稳定性问题，这一点与 softmax 的实现类似。



#### 问题（cross_entropy）：实现交叉熵损失
交付内容：编写一个函数来计算交叉熵损失，该函数接收预测的 logits（$o_i$）和目标值（$x_{i+1}$），并计算交叉熵 $\ell_i = −\log \mathbf{softmax}(o_i)[x_{i+1}]$。你的函数应满足以下要求：

* 减去最大值以保证数值稳定性。
* 尽可能约去 $\log$ 和 $\exp$ 运算，避免数值溢出或下溢。
* 能够处理任意的批量（batch）维度，并对 batch 维度求平均后返回结果。
与第 3.3 节一样，我们假设批量相关的维度始终位于词汇表维度（vocab_size）之前。

In [ ]:
import torch


def softmax(x: torch.Tensor, dim: int) -> torch.Tensor:
    """
    Args:
        x: 输入张量 (batch_size, vocab_size)
    
    数值稳定的 softmax 实现
    通过减去最大值防止 exp 溢出
    """

    # x_max (batch_size, 1) 
    x_max = x.max(dim=dim, keepdim=True)[0]  # 防止 exp(x) 上溢 
    x_exp = torch.exp(x - x_max)  # 稳定的指数计算
    return x_exp / x_exp.sum(dim=dim, keepdim=True)  # 归一化到概率分布


class CrossEntropyLoss:
    def __init__(self, inputs:torch.Tensor, targets:torch.LongTensor):
        """
        Args:
            logits: (..., vocab_size)
            targets: (..., ) 真实标签索引

        初始化交叉熵损失计算器
        """
        self.inputs = inputs  # 模型输出的原始 logits
        self.targets = targets  # 真实标签索引 (long tensor)
        self.vocab_size = inputs.shape[1]  # 词汇表大小
        self.batch_size = inputs.shape[0]  # 批次大小

    def forward(self):
        """
        前向计算交叉熵损失
        步骤：softmax -> 取真实类概率 -> 负对数求和
        """
        y_pred = softmax(self.inputs, dim=1)  # 对每行做 softmax 得预测概率

        # 提取真实标签对应的概率 p = y_pred[i, targets[i]]
        p = y_pred[range(self.batch_size), self.targets]

        # 计算负对数似然并求和
        return -torch.sum(torch.log(p))